In [7]:
import datetime

In [8]:
client_id = '45844a803da249adae64a637787bfd7e'
client_secret = 'dc6e854038474d0ab8ed4faacc9d3080'

In [23]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

In [24]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

strokes_uri = 'spotify:artist:0epOFNiUfyON9EYx7Tpr6V'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id = '45844a803da249adae64a637787bfd7e',
    client_secret = 'dc6e854038474d0ab8ed4faacc9d3080',))

results = spotify.artist_albums(strokes_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

The New Abnormal
Comedown Machine
Comedown Machine
Angles
Angles
First Impressions Of Earth
First Impressions Of Earth
First Impressions of Earth
Room On Fire
Room On Fire
Is This It
Is This It
Is This It
Is This It


In [37]:
import json
import requests
from secrets import spotify_user_id, spotify_token, discover_weekly_id

ImportError: cannot import name 'spotify_user_id' from 'secrets' (/usr/local/Cellar/python@3.8/3.8.5/Frameworks/Python.framework/Versions/3.8/lib/python3.8/secrets.py)

In [34]:
def find_songs(self): 
    print("Finding songs in discover weekly...")
        # Loop through playlist tracks, add them to list

    query = "https://api.spotify.com/v1/playlists/{}/tracks".format(
            discover_weekly_id)

    response = requests.get(query,
                                headers={"Content-Type": "application/json",
                                         "Authorization": "Bearer {}".format(self.spotify_token)})

    response_json = response.json()

    print(response_json)


In [36]:
response_json

NameError: name 'response_json' is not defined